# 1. Trying WhyHow SDK

In [8]:
from whyhow import WhyHow
import os
from dotenv import load_dotenv

load_dotenv("./.env")

client = WhyHow(
    neo4j_user=os.getenv("NEO4J_USERNAME"),
    neo4j_password=os.getenv("NEO4J_PASSWORD"),
    neo4j_url=os.getenv("NEO4J_URL"),
    api_key=os.getenv("WHYHOW_API_KEY"),
    pinecone_api_key=os.getenv("PINECONE_API_KEY"),
)

In [9]:
# Define namespace name
namespace = "hair_salon_reviews"

documents = ["./scrapped_reviews/reviews_selected_small.csv"]

In [10]:
schema_path = "./scrapped_reviews/reviews_selected_schema.json"

In [11]:
# schema = client.graph.generate_schema(documents=documents)

# # save as json file
# with open(schema_path, "w") as f:
#     f.write(schema)

In [12]:
# Add documents to your namespace
documents_response = client.graph.add_documents(
    namespace=namespace, documents=documents
)
print(documents_response)

Your documents are being added in the background.


In [14]:
extracted_graph = client.graph.create_graph_from_csv(namespace, schema_path)
print(extracted_graph)

Your graph creation has started.


In [ ]:
query = "How is Curly Hair Designs reviewed by people?"
query_response = client.graph.query_graph(
    namespace=namespace, query=query, include_triples=True, include_chunks=True
)
print("Query Response:", query_response)

Query Response: namespace='hair_salon_reviews' answer='No context provided. Please add more specific information to the graph, or ask a more specific question based on the entities and relations.' triples=[] chunks=[]


## Problem

I have very little control over how the data is imported to graphs. I'd better use Neo4j Cypher. 

# 2. Using Neo4j Cypher

In [15]:
from neo4j import GraphDatabase

load_dotenv("./.env")

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = os.getenv("NEO4J_URL")
print(f"==>> URI: {URI}")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
print(f"==>> AUTH: {AUTH}")

==>> URI: neo4j+s://5f8021db.databases.neo4j.io
==>> AUTH: ('neo4j', 'FGVrCAoMKNaAWwgoTgdyZHvV9_hSG7Tpbv0oKtINHng')


In [16]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [17]:
records, summary, keys = driver.execute_query(
    "MATCH (p:PLACE_NAME) RETURN p.name AS name",
    database_="neo4j",
)
print(f"==>> records: {records}")
print(f"==>> summary: {summary}")
print(f"==>> keys: {keys}")

/var/folders/9f/n__6xm3123587g3b26ppz2nr0000gn/T/ipykernel_27324/180275419.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


==>> records: [<Record name='Curly Hair Designs'>, <Record name='Lucas Nault Hair Studio'>, <Record name='Ndeg1 SALON'>, <Record name='A-One Hair Salon'>, <Record name='THS The Hair Salon'>, <Record name='Salon Nick'>, <Record name='Richmond Hair Studio'>, <Record name="Claudette's Coiffure & Boutique Inc">, <Record name='Saco Hair Ottawa'>, <Record name='Hair by JamieG'>, <Record name='Pierino Scarfo Salon Westboro'>, <Record name='Shannon Doyle Beauty'>, <Record name='Tony And Chris Hair Studio'>, <Record name="Nevio's Mane Image Hair Studio">, <Record name='Hair World Salon & Spa'>, <Record name='Flawless Hair Salon - Ottawa Balayage Expert'>, <Record name='Francesco Hairstylist & Barber'>, <Record name='Designer 2 Hair Salon'>, <Record name='HairRenos.ca'>, <Record name='Mint Hair Studio'>, <Record name='On Edge Studio'>, <Record name='Precision Styling & Il Paradiso Spa'>, <Record name='Painted Hair co.'>, <Record name='Ombel Salon'>, <Record name='Shampoo Hair Salon'>, <Record na

In [18]:
def add_vendor(transaction, name):
    result = transaction.run("CREATE (a:Vendor {name: $name})", name=name)
    print(f"==>> result: {result}")
    return result.single()

In [21]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session(database="neo4j") as session:
        result = session.execute_write(add_vendor, "MINKI_J")

==>> result: <neo4j._sync.work.result.Result object at 0x10c325ac0>


In [22]:
records = driver.execute_query("MATCH (n:Vendor) RETURN n", database_="neo4j")

/var/folders/9f/n__6xm3123587g3b26ppz2nr0000gn/T/ipykernel_27324/652030622.py:1: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records = driver.execute_query("MATCH (n:Vendor) RETURN n", database_="neo4j")


In [23]:
records

EagerResult(records=[<Record n=<Node element_id='4:4145b493-0fb1-4b32-8981-47da8116c75d:4686' labels=frozenset({'Vendor'}) properties={'name': 'Curly Hair Designs'}>>, <Record n=<Node element_id='4:4145b493-0fb1-4b32-8981-47da8116c75d:4687' labels=frozenset({'Vendor'}) properties={'name': 'Curly Hair Designs'}>>, <Record n=<Node element_id='4:4145b493-0fb1-4b32-8981-47da8116c75d:4688' labels=frozenset({'Vendor'}) properties={'name': 'MINKI_J'}>>], summary=<neo4j._work.summary.ResultSummary object at 0x12353bef0>, keys=['n'])